# CleanUp

In [50]:
%store -r

In [51]:
import boto3
import time
from utility.knowledgebase import delete_iam_role_and_policies 

boto3_session = boto3.Session()
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=boto3_session.region_name)
aoss_client = boto3.client('opensearchserverless')
s3_client = boto3_session.client('s3', region_name=boto3_session.region_name)
iam_client = boto3.client("iam")
lambda_client = boto3.client("lambda")
sns = boto3.client("sns")
sts_client = boto3.client('sts')

region_name = boto3_session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region_name, account_id

('us-east-1', '435212236001')

## Amazon Bedrock Knowledge Base Cleanup

### Delete Bedrock KnowledgeBase data sources

In [31]:
# fetch data source(s)associated with kb
response = bedrock_agent_client.list_data_sources(knowledgeBaseId=kb_id)
data_sources_list = [ds_summary for ds_summary in response['dataSourceSummaries']]

for idx, ds in enumerate(data_sources_list):
    bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=ds["knowledgeBaseId"])
    time.sleep(10)

### Remove KnowledgeBases and OpenSearch Collection


In [32]:
# Fetch kb execution role
response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)
kb_role_name = response['knowledgeBase']['roleArn'].split("/")[-1]

In [33]:

# fetch all attched policies with kb execution role
kb_attached_role_policies_response = iam_client.list_attached_role_policies(
    RoleName=kb_role_name)
kb_attached_role_policies = kb_attached_role_policies_response['AttachedPolicies']

In [34]:
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)
aoss_client.delete_collection(id=collection['createCollectionDetail']['id'])
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

{'ResponseMetadata': {'RequestId': 'b7c1030a-b0e0-4148-b1d1-1f2b1ad24af4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b7c1030a-b0e0-4148-b1d1-1f2b1ad24af4',
   'date': 'Fri, 01 Nov 2024 15:51:07 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Delete role and policies

In [35]:
delete_iam_role_and_policies(kb_role_name, suffix)

0

### S3 Bucket

In [36]:
objects = s3_client.list_objects(Bucket=bucket_name)
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'G52E6R965Y6J4J5G',
  'HostId': 'nVLU5Zg555W+4m+hhKW0ivYE3b8DtUp/bpoAZMfoqyWA6RBJidnhT048mqH1p4rYQbuSIZWUMYc=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'nVLU5Zg555W+4m+hhKW0ivYE3b8DtUp/bpoAZMfoqyWA6RBJidnhT048mqH1p4rYQbuSIZWUMYc=',
   'x-amz-request-id': 'G52E6R965Y6J4J5G',
   'date': 'Fri, 01 Nov 2024 15:51:09 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Amazon Bedrock Agents Clean Up

To delete the agent we need to:

1. Update the action group's to disable it
2. Delete agent action group's
3. Delete Amazon Bedrock Agent
4. Delete AWS Lambda function
5. Delete the created IAM roles and policies

In [52]:
response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id_env,
    actionGroupName=action_group_name_env,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': agent_functions_env
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id_env
)

In [53]:
response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id_employee,
    actionGroupName=action_group_name_employee,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': agent_functions_employee
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id_employee
)


In [54]:
response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id_ticket,
    actionGroupName=action_group_name_ticket,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': agent_functions_ticket
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id_ticket
)

In [55]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [56]:
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

{'ResponseMetadata': {'RequestId': 'c35ea68e-de0d-404a-ad67-701b407d771c',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Fri, 01 Nov 2024 16:17:22 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'c35ea68e-de0d-404a-ad67-701b407d771c'},
  'RetryAttempts': 0}}

In [57]:
agent_bedrock_allow_policy_name

'ticket-assistant-ba-563'

In [58]:
agent_role_name

'AmazonBedrockExecutionRoleForAgents_ticket-assistant'

In [59]:
agent_bedrock_allow_policy_name

'ticket-assistant-ba-563'

In [60]:
agent_bedrock_policy_arn

'arn:aws:iam::435212236001:policy/ticket-assistant-ba-863'

In [61]:
# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name, agent_knowledgebase_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn=f'arn:aws:iam::{account_id}:policy/{db_policy_name}')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_allow_policy_name, db_policy_name, agent_knowledgebase_allow_policy_name]:
    iam_client.delete_policy(
        PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
)

### Delete SNS Topic

In [62]:
response = sns.delete_topic(
    TopicArn=topic_arn
)